In [5]:

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import numpy as np
import pandas as pd
import os
import sys
import argparse
import json
import pickle
import random
import math
import time
import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [6]:
class MatrixFactorization(nn.Module):
    def __init__(self,
                 n_users,
                 n_items,
                 n_users_attr,
                 n_items_attr,
                 embedding_dim=20):
        super().__init__()
        self.user_embeddings = nn.Embedding(n_users, embedding_dim, sparse=True)
        self.item_embeddings = nn.Embedding(n_items, embedding_dim, sparse=True)
        self.user_attribute_embeddings = nn.Embedding(n_users_attr, embedding_dim, sparse=True)
        self.item_attribute_embeddings = nn.Embedding(n_items_attr, embedding_dim, sparse=True)

    def forward(self, user, item, user_attributes, item_attributes):
        user_item_interaction = (self.user_embeddings(user) * self.item_embeddings(item)).sum(1)
        user_attribute_user_interaction = (self.user_attribute_embeddings(user_attributes) * self.user_embeddings(user)).sum(1)
        item_attribute_item_interaction = (self.item_attribute_embeddings(item_attributes) * self.item_embeddings(item)).sum(1)
        return user_item_interaction + user_attribute_user_interaction + item_attribute_item_interaction

    def predict(self, user, item, user_attributes, item_attributes):
        return self.forward(user, item, user_attributes, item_attributes)


class MatrixFactorizationWithBias(nn.Module):
    def __init__(self,
                 n_users,
                 n_items,
                 n_users_attr,
                 n_items_attr,
                 embedding_dim=20):
        super().__init__()
        self.user_embeddings = nn.Embedding(n_users, embedding_dim, sparse=True)
        self.item_embeddings = nn.Embedding(n_items, embedding_dim, sparse=True)
        self.user_attribute_embeddings = nn.Embedding(n_users_attr, embedding_dim, sparse=True)
        self.item_attribute_embeddings = nn.Embedding(n_items_attr, embedding_dim, sparse=True)
        self.user_biases = nn.Embedding(n_users, 1, sparse=True)
        self.item_biases = nn.Embedding(n_items, 1, sparse=True)
        self.user_attribute_biases = nn.Embedding(n_users_attr, 1, sparse=True)
        self.item_attribute_biases = nn.Embedding(n_items_attr, 1, sparse=True)

    def forward(self, user, item, user_attributes, item_attributes):
        user_item_interaction = (self.user_embeddings(user) * self.item_embeddings(item)).sum(1)
        user_attribute_user_interaction = (self.user_attribute_embeddings(user_attributes) * self.user_embeddings(user)).sum(1)
        item_attribute_item_interaction = (self.item_attribute_embeddings(item_attributes) * self.item_embeddings(item)).sum(1)
        user_bias = self.user_biases(user).squeeze()
        item_bias = self.item_biases(item).squeeze()
        user_attribute_bias = self.user_attribute_biases(user_attributes).squeeze()
        item_attribute_bias = self.item_attribute_biases(item_attributes).squeeze()
        return user_item_interaction + user_attribute_user_interaction + item_attribute_item_interaction + user_bias + item_bias + user_attribute_bias + item_attribute_bias

    def predict(self, user, item, user_attributes, item_attributes):
        return self.forward(user, item, user_attributes, item_attributes)

def train(model, train_loader, test_loader, epochs, lr, wd, device, model_dir, model_name, verbose):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    criterion = nn.MSELoss()
    train_losses = []
    test_losses = []
    for epoch in range(epochs):
        train_loss = 0.0
        for i, (user, item, user_attributes, item_attributes, rating) in enumerate(train_loader):
            user = user.to(device)
            item = item.to(device)
            user_attributes = user_attributes.to(device)
            item_attributes = item_attributes.to(device)
            rating = rating.to(device)
            optimizer.zero_grad()
            outputs = model(user, item, user_attributes, item_attributes)
            loss = criterion(outputs, rating)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        train_losses.append(train_loss)
        test_loss = 0.0
        for i, (user, item, user_attributes, item_attributes, rating) in enumerate(test_loader):
            user = user.to(device)
            item = item.to(device)
            user_attributes = user_attributes.to(device)
            item_attributes = item_attributes.to(device)
            rating = rating.to(device)
            outputs = model(user, item, user_attributes, item_attributes)
            loss = criterion(outputs, rating)
            test_loss += loss.item()
        test_loss = test_loss / len(test_loader)
        test_losses.append(test_loss)
        if verbose:
            print('epoch: {}, train loss: {}, test loss: {}'.format(epoch, train_loss, test_loss))
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    torch.save(model.state_dict(), os.path.join(model_dir, model_name))
    return train_losses, test_losses

def evaluate(model, test_loader, device):
    model.eval()
    criterion = nn.MSELoss()
    test_loss = 0.0
    with torch.no_grad():
        for i, (user, item, user_attributes, item_attributes, rating) in enumerate(test_loader):
            user = user.to(device)
            item = item.to(device)
            user_attributes = user_attributes.to(device)
            item_attributes = item_attributes.to(device)
            rating = rating.to(device)
            outputs = model(user, item, user_attributes, item_attributes)
            loss = criterion(outputs, rating)
            test_loss += loss.item()
    test_loss = test_loss / len(test_loader)
    return test_loss

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--train', action='store_true', help='train the model')
    parser.add_argument('--evaluate', action='store_true', help='evaluate the model')
    parser.add_argument('--predict', action='store_true', help='predict the model')
    parser.add_argument('--model_dir', type=str, default='model', help='model directory path')
    parser.add_argument('--model_name', type=str, default='model.pth', help='model name')
    parser.add_argument('--train_data', type=str, default='data/train.csv', help='train data path')
    parser.add_argument('--test_data', type=str, default='data/test.csv', help='test data path')
    parser.add_argument('--user_data', type=str, default='data/user.csv', help='user data path')
    parser.add_argument('--item_data', type=str, default='data/item.csv', help='item data path')
    parser.add_argument('--user_attributes_data', type=str, default='data/user_attributes.csv', help='user attributes data path')
    parser.add_argument('--item_attributes_data', type=str, default='data/item_attributes.csv', help='item attributes data path')
    parser.add_argument('--n_epochs', type=int, default=10, help='the number of epochs')
    parser.add_argument('--batch_size', type=int, default=1024, help='batch size')
    parser.add_argument('--lr', type=float, default=0.01, help='learning rate')
    parser.add_argument('--wd', type=float, default=0.0, help='weight decay')
    parser.add_argument('--embedding_dim', type=int, default=20, help='embedding dimension')
    parser.add_argument('--seed', type=int, default=1234, help='random seed')
    parser.add_argument('--verbose', action='store_true', help='verbose')
    args = parser.parse_args()

    print('Loading data...')
    train_df = pd.read_csv(args.train_data)
    test_df = pd.read_csv(args.test_data)
    user_df = pd.read_csv(args.user_data)
    item_df = pd.read_csv(args.item_data)
    user_attributes_df = pd.read_csv(args.user_attributes_data)
    item_attributes_df = pd.read_csv(args.item_attributes_data)

    print('Preprocessing data...')
    user_ids = sorted(list(set(train_df['user_id'].unique().tolist() + test_df['user_id'].unique().tolist())))
    user2id = {user_id: i for i, user_id in enumerate(user_ids)}
    item_ids = sorted(list(set(train_df['item_id'].unique().tolist() + test_df['item_id'].unique().tolist())))
    item2id = {item_id: i for i, item_id in enumerate(item_ids)}
    user_attribute_ids = sorted(list(set(user_attributes_df['user_attribute'].unique().tolist())))
    user_attribute2id = {user_attribute: i for i, user_attribute in enumerate(user_attribute_ids)}
    item_attribute_ids = sorted(list(set(item_attributes_df['item_attribute'].unique().tolist())))
    item_attribute2id = {item_attribute: i for i, item_attribute in enumerate(item_attribute_ids)}
    train_df['user_id'] = train_df['user_id'].map(user2id)
    train_df['item_id'] = train_df['item_id'].map(item2id)
    test_df['user_id'] = test_df['user_id'].map(user2id)
    test_df['item_id'] = test_df['item_id'].map(item2id)
    user_attributes_df['user_attribute'] = user_attributes_df['user_attribute'].map(user_attribute2id)
    item_attributes_df['item_attribute'] = item_attributes_df['item_attribute'].map(item_attribute2id)
    train_df = train_df.sample(frac=1, random_state=args.seed)
    test_df = test_df.sample(frac=1, random_state=args.seed)
    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)
    user_attributes_df = user_attributes_df.sample(frac=1, random_state=args.seed)
    item_attributes_df = item_attributes_df.sample(frac=1, random_state=args.seed)
    user_attributes_df.reset_index(drop=True, inplace=True)
    item_attributes_df.reset_index(drop=True, inplace=True)
    train_user_ids = train_df['user_id'].values
    train_item_ids = train_df['item_id'].values
    train_user_attributes = user_attributes_df['user_attribute'].values
    train_item_attributes = item_attributes_df['item_attribute'].values
    train_ratings = train_df['rating'].values
    test_user_ids = test_df['user_id'].values
    test_item_ids = test_df['item_id'].values
    test_user_attributes = user_attributes_df['user_attribute'].values
    test_item_attributes = item_attributes_df['item_attribute'].values
    test_ratings = test_df['rating'].values
    train_dataset = TensorDataset(torch.LongTensor(train_user_ids),
                                  torch.LongTensor(train_item_ids),
                                  torch.LongTensor(train_user_attributes),
                                  torch.LongTensor(train_item_attributes),
                                  torch.FloatTensor(train_ratings))
    test_dataset = TensorDataset(torch.LongTensor(test_user_ids),
                                    torch.LongTensor(test_item_ids),
                                    torch.LongTensor(test_user_attributes),
                                    torch.LongTensor(test_item_attributes),
                                    torch.FloatTensor(test_ratings))
    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

    print('Building model...')
    model = MatrixFactorizationWithBias(n_users=len(user_ids),
                                        n_items=len(item_ids),
                                        n_users_attr=len(user_attribute_ids),
                                        n_items_attr=len(item_attribute_ids),
                                        embedding_dim=args.embedding_dim)
    
    if args.train:
        print('Training model...')
        train_losses, test_losses = train(model, train_loader, test_loader, args.n_epochs, args.lr, args.wd, device, args.model_dir, args.model_name, args.verbose)
        if args.verbose:
            print('Saving losses...')
        with open(os.path.join(args.model_dir, 'train_losses.pkl'), 'wb') as f:
            pickle.dump(train_losses, f)
        with open(os.path.join(args.model_dir, 'test_losses.pkl'), 'wb') as f:
            pickle.dump(test_losses, f)
    if args.evaluate:
        print('Evaluating model...')
        model.load_state_dict(torch.load(os.path.join(args.model_dir, args.model_name)))
        test_loss = evaluate(model, test_loader, device)
        print('test loss: {}'.format(test_loss))
    if args.predict:
        print('Predicting model...')
        model.load_state_dict(torch.load(os.path.join(args.model_dir, args.model_name)))
        model.eval()
        with torch.no_grad():
            for i, (user, item, user_attributes, item_attributes, rating) in enumerate(test_loader):
                user = user.to(device)
                item = item.to(device)
                user_attributes = user_attributes.to(device)
                item_attributes = item_attributes.to(device)
                rating = rating.to(device)
                outputs = model(user, item, user_attributes, item_attributes)
                print('user: {}, item: {}, user_attributes: {}, item_attributes: {}, rating: {}, predicted rating: {}'.format(user, item, user_attributes, item_attributes, rating, outputs))
    
if __name__ == '__main__':
    main()

    


usage: ipykernel_launcher.py [-h] [--train] [--evaluate] [--predict]
                             [--model_dir MODEL_DIR] [--model_name MODEL_NAME]
                             [--train_data TRAIN_DATA] [--test_data TEST_DATA]
                             [--user_data USER_DATA] [--item_data ITEM_DATA]
                             [--user_attributes_data USER_ATTRIBUTES_DATA]
                             [--item_attributes_data ITEM_ATTRIBUTES_DATA]
                             [--n_epochs N_EPOCHS] [--batch_size BATCH_SIZE]
                             [--lr LR] [--wd WD]
                             [--embedding_dim EMBEDDING_DIM] [--seed SEED]
                             [--verbose]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/ssdasgupta/Library/Jupyter/runtime/kernel-v2-913BGHljbuamyuX.json


SystemExit: 2

/Users/ssdasgupta/miniconda3/envs/boxrec/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
